In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix

# from sklearn.preprocessing import StandardScaler, OneHotEncoder


# from sklearn.neural_network import MLPRegressor, MLPClassifier


# from sklearn.model_selection import train_test_split
import re

from sklearn.linear_model import Ridge 

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import os


# import requests
# from bs4 import BeautifulSoup as soup

# from selenium import webdriver


from acer_login import create_directory, log_n_pull


Checking for mac64 chromedriver:76.0.3809.68 in cache
Driver found in /Users/austinader/.wdm/chromedriver/76.0.3809.68/mac64/chromedriver


In [2]:
chemistry = ['carbon', 'manganese', 'sulfur', 'phosphorus', 'silicon', 'niobium', 'vanadium', 'aluminium']

desired_directory = '/Users/austinader/ak_Steel_Scrape/raw_acermital_dfs'


## Import/clean ak_df...training data

In [3]:
ak_df = pd.read_csv('/Users/austinader/ak_Steel_Scrape/clean_ak_df', dtype = {'defect_code': 'object'}, float_precision='round_trip',
                            index_col = 'index')



ak_df.drop(columns = ['location', 'defect', 'linear feat', 'defect_code'], inplace = True)

ak_df.dropna(inplace = True)

# ak_df = ak_df.loc[:, intersect_vars]

In [4]:
ak_df.head()

price  carbon  manganese  phosphorus  sulfur  silicon  aluminium  \
index                                                                           
684778-06AA   30.0  0.0038      0.332       0.015  0.0006    0.300     0.0069   
681110-02A    30.0  0.0038      0.332       0.015  0.0006    0.300     0.0069   
686359-01     30.0  0.0052      0.431       0.016  0.0006    0.317     0.0074   
684481-04BA   30.0  0.0096      0.340       0.023  0.0013    0.390     0.0080   
684592-01AA   30.0  0.0100      0.310       0.023  0.0015    0.370     0.0080   

             niobium  vanadium   gauge   width   weight  
index                                                    
684778-06AA    0.005     0.054  0.0411  45.347  21350.0  
681110-02A     0.005     0.054  0.0572  48.358  38470.0  
686359-01      0.001     0.058  0.0370  48.000  36870.0  
684481-04BA    0.017     0.066  0.0470  48.858   9140.0  
684592-01AA    0.024     0.065  0.0566  44.582  11850.0

## attempting to concat two ak steel df

In [12]:
# creating function that reads all excels in a df and concatenates

def acer_excel_reader():

    list_of_dfs = []
    for i in [f for f in os.listdir(os.getcwd()) if not f.startswith('.')]:
         list_of_dfs.append(pd.read_html(str(i), skiprows=1, header=0, index_col='Piece ID / Ref#')[0])

    return pd.concat(list_of_dfs)

acer_df = acer_excel_reader()


In [14]:
acer_df.head()

Minor Product              Business Unit Category  \
Piece ID / Ref#                                                     
373015 / 753586  ALL OTHER CR  USA - Indiana Harbor East  Salvage   
373016 / 669727  ALL OTHER CR  USA - Indiana Harbor East  Salvage   
373017 / 669727  ALL OTHER CR  USA - Indiana Harbor East  Salvage   
372970 / 753339  ALL OTHER CR  USA - Indiana Harbor East  Salvage   
372976 / 753339  ALL OTHER CR  USA - Indiana Harbor East  Salvage   

                           Gauge   Width  Weight  \
Piece ID / Ref#                                    
373015 / 753586  0.0190 x 0.0190  47.875   11000   
373016 / 669727  0.0190 x 0.0190  47.875   11010   
373017 / 669727  0.0190 x 0.0190  47.875   11010   
372970 / 753339  0.0190 x 0.0190  47.875   11020   
372976 / 753339  0.0190 x 0.0190  47.875   11020   

                                      Location Grade  \
Piece ID / Ref#                                        
373015 / 753586  AM-TAYLOR COIL LORDSTOWN , 34   CSB   
373016 / 669727  AM-TAYLOR COIL LORDSTOWN , 34   CSB   
373017 / 669727  AM-TAYLOR COIL LORDSTOWN , 34   CSB   
372970 / 753339  AM-TAYLOR COIL LORDSTOWN , 34   CSB   
372976 / 753339  AM-TAYLOR COIL LORDSTOWN , 34   CSB   

                                                         Chemistry  \
Piece ID / Ref#                                                      
373015 / 753586  C: .0328 MN: .1764 S: .0058 P: .0055 SI: .0077...   
373016 / 669727  C: .0352 MN: .2325 S: .0078 P: .0074 SI: .0171...   
373017 / 669727  C: .0352 MN: .2325 S: .0078 P: .0074 SI: .0171...   
372970 / 753339  C: .0319 MN: .1817 S: .0079 P: .0065 SI: .0081...   
372976 / 753339  C: .0319 MN: .1817 S: .0079 P: .0065 SI: .0081...   

                                         Comment   Defect  Trans#  
Piece ID / Ref#                                                    
373015 / 753586  LIGHT CHATTER AND SPECKLED RUST  CHATTER     NaN  
373016 / 669727  LIGHT CHATTER AND SPECKLED RUST  CHATTER     NaN  
373017 / 669727  LIGHT CHATTER AND SPECKLED RUST  CHATTER     NaN  
372970 / 753339  LIGHT CHATTER AND SPECKLED RUST  CHATTER     NaN  
372976 / 753339  LIGHT CHATTER AND SPECKLED RUST  CHATTER     NaN

## Import/clean acer_df, not containing target price

In [15]:
# acer_df = pd.read_excel('test_acermital_df.xlsx', skiprows=6, index_col='Piece ID / Ref#')

acer_df.rename(columns = {'Gauge':'gauge', 'Width':'width', 'Weight':'weight'}, inplace = True)


acer_df.drop(columns = ['Minor Product', 'Business Unit', 'Category', 'Location', 'Grade', 'Comment', 'Defect', 'Trans#'], inplace = True)


def gauge_fix():
    gauge_data = acer_df['gauge'].values
    
    pretty_gauges = []
    for i in gauge_data:
        pretty_gauges.append(float(i.split()[0]))
    
    return pretty_gauges


acer_df['gauge'] = gauge_fix()


def chem_fix():
    global acer_df
    chem_list = acer_df['Chemistry'].values.tolist()
    chem_dict = {k:None for k in chemistry }


    chem_clean = []
    for i in chem_list:
        chem_clean.append(i.replace(":",": ").split())


    carbon = []
    manganese = []
    sulfur = []
    phosphorus = []
    silicon = []
    niobium = []
    vanadium = []
    aluminium = []

    for auction in chem_clean:
        carbon.append(float(auction[auction.index('C:') + 1]))
        manganese.append(float(auction[auction.index('MN:') + 1]))
        sulfur.append(float(auction[auction.index('S:') + 1]))
        phosphorus.append(float(auction[auction.index('P:') + 1]))
        silicon.append(float(auction[auction.index('SI:') + 1]))
        niobium.append(float(auction[auction.index('CB:') + 1]))
        vanadium.append(float(auction[auction.index('V:') + 1]))
        aluminium.append(float(auction[auction.index('AL:') + 1]))


    chem_dict['carbon'] = carbon
    chem_dict['manganese'] = manganese
    chem_dict['sulfur'] = sulfur
    chem_dict['phosphorus'] = phosphorus
    chem_dict['silicon'] = silicon
    chem_dict['niobium'] = niobium
    chem_dict['vanadium'] = vanadium
    chem_dict['aluminium'] = aluminium

    acer_df['carbon'] = chem_dict['carbon'] 
    acer_df['manganese'] = chem_dict['manganese']
    acer_df['sulfur'] = chem_dict['sulfur']
    acer_df['phosphorus'] = chem_dict['phosphorus']
    acer_df['silicon'] = chem_dict['silicon']
    acer_df['niobium'] = chem_dict['niobium']
    acer_df['vanadium'] = chem_dict['vanadium']
    acer_df['aluminium'] = chem_dict['aluminium']
    
    acer_df.drop(columns = ['Chemistry'], inplace = True)
    
    acer_df = acer_df.reindex(columns = ak_df.columns.values[1:])
    
    
    


chem_fix()


## splitting data

In [17]:
y_data = ak_df.loc[:, 'price'].values.reshape(-1,1)

X_data = ak_df.iloc[:, 1:].values.reshape(len(ak_df),-1)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=42)



## Training Ridge model on ak_df

In [18]:
ridge = Ridge(alpha = .01)

ridge.fit(X_train, y_train)

ridge.score(X_test, y_test)

np.sqrt(mean_squared_error(y_test, ridge.predict(X_test)))






7.316520845494016

## running/appending model on acer_df

In [19]:
acer_predicts = np.ravel(ridge.predict(acer_df.values.reshape(len(acer_df), -1)))

In [20]:
acer_pred_df = acer_df.copy()

acer_pred_df['predictions'] = acer_predicts

In [21]:
acer_pred_df.to_excel('acermital_predict_test.xlsx')

In [22]:
acer_pred_df

carbon  manganese  phosphorus  sulfur  silicon  \
Piece ID / Ref#                                                        
373015 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
373016 / 669727       0.0352     0.2325      0.0074  0.0078   0.0171   
373017 / 669727       0.0352     0.2325      0.0074  0.0078   0.0171   
372970 / 753339       0.0319     0.1817      0.0065  0.0079   0.0081   
372976 / 753339       0.0319     0.1817      0.0065  0.0079   0.0081   
372991 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
372988 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
372995 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
373001 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
372986 / 753398       0.0297     0.2052      0.0063  0.0066   0.0158   
373003 / 669727       0.0352     0.2325      0.0074  0.0078   0.0171   
372993 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
373004 / 753586       0.0328     0.1764      0.0055  0.0058   0.0077   
372966 / 753339       0.0319     0.1817      0.0065  0.0079   0.0081   
372985 / 760312       0.0324     0.1979      0.0077  0.0081   0.0055   
373005 / 753398       0.0297     0.2052      0.0063  0.0066   0.0158   
372984 / 753101       0.0316     0.1950      0.0063  0.0073   0.0108   
373007 / 753101       0.0316     0.1950      0.0063  0.0073   0.0108   
373008 / 753101       0.0316     0.1950      0.0063  0.0073   0.0108   
373009 / 669727       0.0352     0.2325      0.0074  0.0078   0.0171   
373011 / 769991       0.0320     0.2063      0.0052  0.0085   0.0070   
373020 / 769991       0.0320     0.2063      0.0052  0.0085   0.0070   
373021 / 753398       0.0297     0.2052      0.0063  0.0066   0.0158   
372971 / 753339       0.0319     0.1817      0.0065  0.0079   0.0081   
372964 / 753339       0.0319     0.1817      0.0065  0.0079   0.0081   
372965 / 753339       0.0319     0.1817      0.0065  0.0079   0.0081   
372996 / 760226       0.0328     0.2123      0.0054  0.0054   0.0210   
372978 / 753623       0.0414     0.1960      0.0050  0.0060   0.0055   
372994 / 753398       0.0297     0.2052      0.0063  0.0066   0.0158   
372999 / 760226       0.0328     0.2123      0.0054  0.0054   0.0210   
...                      ...        ...         ...     ...      ...   
970-92284 / 956572    0.0289     0.1833      0.0085  0.0085   0.0132   
T017964 / 851254      0.0087     0.6068      0.0054  0.0079   0.5362   
4276313.010 / 953644  0.0021     0.0958      0.0040  0.0096   0.0057   
4139544.010 / 866787  0.0020     0.1024      0.0039  0.0102   0.0074   
970-91952 / 961844    0.0017     0.0995      0.0054  0.0061   0.0052   
877862001 / 866785    0.0018     0.1004      0.0040  0.0105   0.0071   
920-40804 / 869949    0.0312     0.1912      0.0081  0.0103   0.0135   
920-40803 / 841726    0.0500     0.2160      0.0080  0.0039   0.0270   
920-40805 / 960955    0.0302     0.2044      0.0058  0.0056   0.0058   
920-42901 / 960955    0.0302     0.2044      0.0058  0.0056   0.0058   
970-92149 / 852892    0.0296     0.1885      0.0092  0.0119   0.0119   
970-91840 / 869508    0.0335     0.2138      0.0070  0.0076   0.0133   
920-41081 / 962754    0.0282     0.1960      0.0046  0.0067   0.0177   
970-91097 / 961798    0.0291     0.1891      0.0071  0.0076   0.0056   
X219019 / 955452      0.0317     0.1857      0.0049  0.0103   0.0113   
875875900 / 786227    0.1000     2.0500      0.0120  0.0030   0.6000   
945509301 / 842275    0.0877     0.9980      0.0080  0.0004   0.2900   
945509302 / 842275    0.0877     0.9980      0.0080  0.0004   0.2900   
R1078227 / 832329     0.0969     1.7270      0.0110  0.0017   0.2890   
R1077638 / 832329     0.0969     1.7270      0.0110  0.0017   0.2890   
40359 / 886579        0.1000     2.0400      0.0110  0.0030   0.6000   
T008256 / 615268      0.1508     2.1026      0.0086  0.0008   0.6430   
391510902 / 615286    0.1506     2.0782      0.0096  0.0008   